In [ ]:
import ee
import pandas as pd
import os
from datetime import datetime

# Initialize Earth Engine
ee.Initialize()

# Directory containing CSV files
input_dir = "Katihar/Katihar_Chunks/"
output_dir = "Katihar/Processed/"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define start and end dates
startDate = ee.Date('2021-04-01')
endDate = ee.Date('2021-09-30')
daysStep = 5
nSteps = endDate.difference(startDate, 'day').divide(daysStep).ceil().int()

# Create date ranges
def create_date_range(n):
    n = ee.Number(n)
    return ee.Dictionary({
        'start': startDate.advance(n.multiply(daysStep), 'day'),
        'end': startDate.advance(n.multiply(daysStep).add(daysStep), 'day')
    })

results = ee.List.sequence(0, nSteps.subtract(1)).map(create_date_range)

# Sentinel-2 NDVI calculation
s2 = ee.ImageCollection('COPERNICUS/S2')
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

s2 = s2.map(add_ndvi)

# Function to check if Sentinel-2 data is available
def check_data_availability(point, start, end):
    filtered = s2.filterDate(start, end).filterBounds(point)
    count = filtered.size().getInfo()
    return count > 0  # True if images exist, False otherwise

# Function to sample NDVI values
def sample_ndvi(step, point):
    start = ee.Date(step.get('start'))
    end = ee.Date(step.get('end'))
    filtered = s2.filterDate(start, end).filterBounds(point)

    # Compute NDVI
    ndvi_image = filtered.mean().addBands(filtered.mean().normalizedDifference(['B8', 'B4']).rename('NDVI'))

    # Reduce region
    ndvi_value = ndvi_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=10,
        bestEffort=True
    ).get('NDVI')

    return ee.Algorithms.If(ndvi_value, ndvi_value, -9999)  # Return -9999 if NDVI is missing

# Process each CSV file
for file_name in os.listdir(input_dir):
    if file_name.endswith(".csv"):
        file_path = os.path.join(input_dir, file_name)
        output_path = os.path.join(output_dir, file_name)

        print(f"Processing {file_name}...")

        # Load CSV
        data = pd.read_csv(file_path)
        ndvi_timeseries = []

        # Process each row
        for idx, row in data.iterrows():
            lat, lon = row['Latitude'], row['Longitude']
            print(f'  Latitude: {lat}, Longitude: {lon}')
            point = ee.Geometry.Point(lon, lat)

            # Check if data exists
            if not check_data_availability(point, startDate, endDate):
                print(f"  No Sentinel-2 data for {lat}, {lon}. Skipping...")
                ndvi_values = [-9999] * nSteps.getInfo()
            else:
                try:
                    ndvi_values = results.map(lambda step: sample_ndvi(ee.Dictionary(step), point)).getInfo()
                except Exception as e:
                    print(f"  Error processing {lat}, {lon}: {e}")
                    ndvi_values = [-9999] * nSteps.getInfo()

            ndvi_timeseries.append(ndvi_values)

        # Store NDVI values and retain metadata
        data['NDVI_TimeSeries'] = ndvi_timeseries

        # Save the output
        data.to_csv(output_path, index=False)
        print(f"Saved results to {output_path}\n")

print("Processing completed for all CSV files.")


C:\Users\prath\anaconda3\Lib\site-packages\ee\deprecation.py:202: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Processing Katihar_1.csv...
  Latitude: 25.38320091, Longitude: 87.6794794
  Latitude: 25.38320091, Longitude: 87.67956923
  Latitude: 25.38320091, Longitude: 87.67965906
  Latitude: 25.38320091, Longitude: 87.6797489
  Latitude: 25.38320091, Longitude: 87.67983873
  Latitude: 25.38320091, Longitude: 87.67992856
  Latitude: 25.38320091, Longitude: 87.68001839
  Latitude: 25.38320091, Longitude: 87.68010822
  Latitude: 25.38320091, Longitude: 87.68019805
  Latitude: 25.38320091, Longitude: 87.68028789
  Latitude: 25.38320091, Longitude: 87.68037772
  Latitude: 25.38320091, Longitude: 87.68046755
  Latitude: 25.38320091, Longitude: 87.73194101
  Latitude: 25.38320091, Longitude: 87.73203085
  Latitude: 25.38320091, Longitude: 87.73212068
  Latitude: 25.38320091, Longitude: 87.73221051
  Latitude: 25.38320091, Longitude: 87.73230034
  Latitude: 25.38329074, Longitude: 87.6794794
  Latitude: 25.38329074, Longitude: 87.67956923
  Latitude: 25.38329074, Longitude: 87.67965906
  Latitude: 25.

  Latitude: 25.38841114, Longitude: 87.72888674
  Latitude: 25.38841114, Longitude: 87.72897657
  Latitude: 25.38841114, Longitude: 87.72960539
  Latitude: 25.38841114, Longitude: 87.72969523
  Latitude: 25.38841114, Longitude: 87.72978506
  Latitude: 25.38841114, Longitude: 87.72987489
  Latitude: 25.38850097, Longitude: 87.72825792
  Latitude: 25.38850097, Longitude: 87.72834775
  Latitude: 25.38850097, Longitude: 87.72843758
  Latitude: 25.38850097, Longitude: 87.72852742
  Latitude: 25.38850097, Longitude: 87.72861725
  Latitude: 25.38850097, Longitude: 87.72870708
  Latitude: 25.38850097, Longitude: 87.72879691
  Latitude: 25.38850097, Longitude: 87.72888674
  Latitude: 25.38850097, Longitude: 87.72897657
  Latitude: 25.38850097, Longitude: 87.72906641
  Latitude: 25.38850097, Longitude: 87.72951556
  Latitude: 25.38850097, Longitude: 87.72960539
  Latitude: 25.38850097, Longitude: 87.72969523
  Latitude: 25.38850097, Longitude: 87.72978506
  Latitude: 25.38850097, Longitude: 87.7

  Latitude: 25.43386589, Longitude: 87.69555925
  Latitude: 25.43395572, Longitude: 87.69537958
  Latitude: 25.43395572, Longitude: 87.69546941
  Latitude: 25.43395572, Longitude: 87.69555925
  Latitude: 25.43404556, Longitude: 87.69546941
  Latitude: 25.43404556, Longitude: 87.69555925
  Latitude: 25.43413539, Longitude: 87.69546941
  Latitude: 25.43413539, Longitude: 87.69555925
  Latitude: 25.43413539, Longitude: 87.69564908
  Latitude: 25.43413539, Longitude: 87.69573891
  Latitude: 25.43422522, Longitude: 87.69546941
  Latitude: 25.43422522, Longitude: 87.69555925
  Latitude: 25.43422522, Longitude: 87.69564908
  Latitude: 25.43422522, Longitude: 87.69573891
  Latitude: 25.43431505, Longitude: 87.69546941
  Latitude: 25.43431505, Longitude: 87.69555925
  Latitude: 25.43431505, Longitude: 87.69564908
  Latitude: 25.43431505, Longitude: 87.69573891
  Latitude: 25.43440488, Longitude: 87.69555925
  Latitude: 25.43440488, Longitude: 87.69564908
  Latitude: 25.43440488, Longitude: 87.6

  Latitude: 25.43871679, Longitude: 87.69924234
  Latitude: 25.43871679, Longitude: 87.69933217
  Latitude: 25.43871679, Longitude: 87.699422
  Latitude: 25.43871679, Longitude: 87.69951183
  Latitude: 25.43871679, Longitude: 87.69960166
  Latitude: 25.43871679, Longitude: 87.6996915
  Latitude: 25.43871679, Longitude: 87.69978133
  Latitude: 25.43871679, Longitude: 87.69987116
  Latitude: 25.43880663, Longitude: 87.69870335
  Latitude: 25.43880663, Longitude: 87.69879318
  Latitude: 25.43880663, Longitude: 87.69888301
  Latitude: 25.43880663, Longitude: 87.69897284
  Latitude: 25.43880663, Longitude: 87.69906267
  Latitude: 25.43880663, Longitude: 87.69915251
  Latitude: 25.43880663, Longitude: 87.69924234
  Latitude: 25.43880663, Longitude: 87.69933217
  Latitude: 25.43880663, Longitude: 87.699422
  Latitude: 25.43880663, Longitude: 87.69951183
  Latitude: 25.43880663, Longitude: 87.69960166
  Latitude: 25.43889646, Longitude: 87.69870335
  Latitude: 25.43889646, Longitude: 87.698793

...

  Latitude: 25.60490512, Longitude: 87.35644523
  Latitude: 25.60490512, Longitude: 87.35653506
  Latitude: 25.60490512, Longitude: 87.35662489
  Latitude: 25.60490512, Longitude: 87.35671472
  Latitude: 25.60499495, Longitude: 87.35599607
  Latitude: 25.60499495, Longitude: 87.3560859
  Latitude: 25.60499495, Longitude: 87.35617573
  Latitude: 25.60499495, Longitude: 87.35626556
  Latitude: 25.60499495, Longitude: 87.35635539
  Latitude: 25.60499495, Longitude: 87.35644523
  Latitude: 25.60499495, Longitude: 87.35653506
  Latitude: 25.60499495, Longitude: 87.35662489
  Latitude: 25.60499495, Longitude: 87.35671472
  Latitude: 25.60499495, Longitude: 87.35680455
  Latitude: 25.60508479, Longitude: 87.35599607
  Latitude: 25.60508479, Longitude: 87.3560859
  Latitude: 25.60508479, Longitude: 87.35617573
  Latitude: 25.60508479, Longitude: 87.35626556
  Latitude: 25.60508479, Longitude: 87.35635539
  Latitude: 25.60508479, Longitude: 87.35644523
  Latitude: 25.60508479, Longitude: 87.356

  Latitude: 25.60706108, Longitude: 87.32087194
  Latitude: 25.60706108, Longitude: 87.32096177
  Latitude: 25.60706108, Longitude: 87.3210516
  Latitude: 25.60715091, Longitude: 87.32042278
  Latitude: 25.60715091, Longitude: 87.32051261
  Latitude: 25.60715091, Longitude: 87.32060245
  Latitude: 25.60715091, Longitude: 87.32069228
  Latitude: 25.60715091, Longitude: 87.32078211
  Latitude: 25.60715091, Longitude: 87.32087194
  Latitude: 25.60715091, Longitude: 87.32096177
  Latitude: 25.60724074, Longitude: 87.32060245
  Latitude: 25.60724074, Longitude: 87.32069228
  Latitude: 25.60724074, Longitude: 87.32078211
  Latitude: 25.60724074, Longitude: 87.32087194
  Latitude: 25.60751024, Longitude: 87.32239908
  Latitude: 25.60751024, Longitude: 87.32248891
  Latitude: 25.60751024, Longitude: 87.32257874
  Latitude: 25.60751024, Longitude: 87.32266857
  Latitude: 25.60751024, Longitude: 87.3227584
  Latitude: 25.60751024, Longitude: 87.32284823
  Latitude: 25.60751024, Longitude: 87.322

  Latitude: 25.60903737, Longitude: 87.32006346
  Latitude: 25.60903737, Longitude: 87.32015329
  Latitude: 25.60903737, Longitude: 87.32024312
  Latitude: 25.60903737, Longitude: 87.32033295
  Latitude: 25.6091272, Longitude: 87.31916514
  Latitude: 25.6091272, Longitude: 87.31925497
  Latitude: 25.6091272, Longitude: 87.3193448
  Latitude: 25.6091272, Longitude: 87.31943464
  Latitude: 25.6091272, Longitude: 87.31952447
  Latitude: 25.6091272, Longitude: 87.3196143
  Latitude: 25.6091272, Longitude: 87.31970413
  Latitude: 25.6091272, Longitude: 87.31979396
  Latitude: 25.6091272, Longitude: 87.31988379
  Latitude: 25.6091272, Longitude: 87.31997362
  Latitude: 25.6091272, Longitude: 87.32006346
  Latitude: 25.6091272, Longitude: 87.32015329
  Latitude: 25.6091272, Longitude: 87.32024312
  Latitude: 25.6091272, Longitude: 87.32033295
  Latitude: 25.60921704, Longitude: 87.31916514
  Latitude: 25.60921704, Longitude: 87.31925497
  Latitude: 25.60921704, Longitude: 87.3193448
  Latitud

  Latitude: 25.61191198, Longitude: 87.3164702
  Latitude: 25.61191198, Longitude: 87.31656003
  Latitude: 25.61191198, Longitude: 87.31664986
  Latitude: 25.61191198, Longitude: 87.31673969
  Latitude: 25.61191198, Longitude: 87.31682952
  Latitude: 25.61191198, Longitude: 87.31691935
  Latitude: 25.61191198, Longitude: 87.31700918
  Latitude: 25.61191198, Longitude: 87.31709902
  Latitude: 25.61200181, Longitude: 87.31584137
  Latitude: 25.61200181, Longitude: 87.31593121
  Latitude: 25.61200181, Longitude: 87.31602104
  Latitude: 25.61200181, Longitude: 87.31611087
  Latitude: 25.61200181, Longitude: 87.3162007
  Latitude: 25.61200181, Longitude: 87.31629053
  Latitude: 25.61200181, Longitude: 87.31638036
  Latitude: 25.61200181, Longitude: 87.3164702
  Latitude: 25.61200181, Longitude: 87.31656003
  Latitude: 25.61200181, Longitude: 87.31664986
  Latitude: 25.61200181, Longitude: 87.31673969
  Latitude: 25.61200181, Longitude: 87.31682952
  Latitude: 25.61200181, Longitude: 87.3169

  Latitude: 25.62897997, Longitude: 87.36974029
  Latitude: 25.6290698, Longitude: 87.3694708
  Latitude: 25.6290698, Longitude: 87.36956063
  Latitude: 25.6290698, Longitude: 87.36965046
  Latitude: 25.6290698, Longitude: 87.36974029
  Latitude: 25.62915963, Longitude: 87.3694708
  Latitude: 25.62915963, Longitude: 87.36956063
  Latitude: 25.62915963, Longitude: 87.36965046
  Latitude: 25.62915963, Longitude: 87.36974029
  Latitude: 25.62924947, Longitude: 87.36956063
  Latitude: 25.62924947, Longitude: 87.36965046
  Latitude: 25.62924947, Longitude: 87.36974029
  Latitude: 25.62924947, Longitude: 87.36983012
  Latitude: 25.6293393, Longitude: 87.36956063
  Latitude: 25.6293393, Longitude: 87.36965046
  Latitude: 25.6293393, Longitude: 87.36974029
  Latitude: 25.6293393, Longitude: 87.36983012
  Latitude: 25.62942913, Longitude: 87.36956063
  Latitude: 25.62942913, Longitude: 87.36965046
  Latitude: 25.62942913, Longitude: 87.36974029
  Latitude: 25.62942913, Longitude: 87.36983012
  